### 7.1.4  检测噪声中的失真信号

噪声的存在通常使确定信号的频谱成分变得困难。在这种情况下，频率分析可以提供帮助。
例如，考虑引入三阶失真的非线性放大器的仿真输出。
创建一个输入信号，以 3.6 kHz 采样的 180 Hz 单位振幅正弦波。生成 10000 个样本。
在输入中添加单位方差白噪声。使用三阶多项式对放大器进行建模。将输入信号通过放大器。绘制输出的一部分。为了进行比较，绘制纯正弦曲线的输出

In [1]:
using Random
using TyPlot
using Polynomials
using TyMath
using TySignalProcessing

# MATLAB variables
N = 10000
n = 0:N-1
fs = 3600
f0 = 180
t = n ./ fs
y = sin.(2 * pi * f0 .* t)

# 设置随机数生成器
Random.seed!(0)
noise = randn(size(y))

# MATLAB的polyval函数转换
dispol = [0.5, 0.75, 1, 0]
out = polyval(dispol, y .+ noise)

ns = 300:500

# 绘图
figure()
plot(t[ns], out[ns], t[ns], polyval(dispol, y[ns]))
xlabel("Time (s)")
ylabel("Signals")
legend(["With white noise", "No white noise"])


PyObject <objects.mw_legend.CLegend object at 0x0000023FBF01B348>

pwelch函数用于计算和绘制输出的功率谱密度。

In [2]:
# 计算Welch功率谱密度估计
segmentLength = 512  # 设定段长度
pxx, f = pwelch(out, segmentLength; nargout = 2)

# 绘制Welch功率谱密度估计结果
figure()
plot(f .* fs / pi, 10 * log10.(pxx))
grid("on")
title("Welch Power Spectral Density Estimate")
xlabel("Frequency (Hz)")
ylabel("Power/Frequency (dB/Hz)")
xlim([0, fs / 2])

(0.0, 1800.0)

由于放大器引入了三阶失真，因此输出信号应具有：
1、DC（零频）组件;
2、与输入频率相同的基波分量，180 Hz;
3、两个谐波 -- 频率分量是输入频率的两倍和三倍，360 和 540 Hz。
验证输出是否符合三次非线性的预期值。

In [8]:
figure()
plot(f .* fs / pi, 10 * log10.(pxx),locs, 10 * log10.(pks), "or")
grid("on")
title("Welch Power Spectral Density Estimate")
xlabel("Frequency (Hz)")
ylabel("Power/Frequency (dB/Hz)")
legend(["PSD", "Frequency Components"])
xlim([0, fs / 2])

UndefVarError: UndefVarError: `locs` not defined

In [4]:
# 排序频率成分
components = sort(vcat(locs, f0 .* (0:3)))

UndefVarError: UndefVarError: `locs` not defined

使用不同的段长度计算Welch功率谱密度估计：
pwelch工作原理是将信号划分为重叠的段，计算每个段的周期图，然后取平均值。默认情况下，该函数使用 8 个重叠 50% 的段。对于 10000 个样本，这相当于每个段 2222 个样本。
将信号分成更短的段可产生更多的平均值。周期图更平滑，但分辨率较低。无法区分高次谐波。

In [5]:
# 段长度为222
segmentLength1 = 222
pxx1, f1 = pwelch(out, segmentLength1; nargout = 2)
# 绘制Welch功率谱密度估计结果
figure()
plot(f1 .* fs / pi, 10 * log10.(pxx1))
grid("on")
title("Welch Power Spectral Density Estimate (Segment Length = 222)")
xlabel("Frequency (Hz)")
ylabel("Power/Frequency (dB/Hz)")
xlim([0, fs / 2])

(0.0, 1800.0)

将信号分成更长的段可以提高分辨率，还可以提高随机性。信号和谐波精确地位于预期位置。但是，至少存在一个杂散高频峰值，其功率高于高次谐波。

In [6]:
# 段长度为4444
segmentLength2 = 4444
pxx2, f2 = pwelch(out, segmentLength2; nargout = 2)

# 绘制Welch功率谱密度估计结果
figure()
plot(f2 .* fs / pi, 10 * log10.(pxx2))
grid("on")
title("Welch Power Spectral Density Estimate (Segment Length = 4444)")
xlabel("Frequency (Hz)")
ylabel("Power/Frequency (dB/Hz)")
xlim([0, fs / 2])

(0.0, 1800.0)